In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('../src_jobs/')

In [2]:
import pickle
from itertools import repeat
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch
from artifact import Saw
from sliding_window_detector import SlidingWindowTransformerDetector, ConvolutionalSlidingWindowDetector, SlidingWindowLinearDetector
from mask_detector import WindowLinearDetector, WindowTransformerDetector, ConvolutionDetector

from data import RealisticArtifactDataset, CachedArtifactDataset, TestArtifactDataset, CenteredArtifactDataset

/opt/miniconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.set_grad_enabled(False)
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

In [4]:
test_width = 512

test_path = Path("/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/test_label_CinCECGTorso512.pkl")
test = CachedArtifactDataset(file=test_path)

val_path = Path("/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/val_SW_noCiECGT512.pkl")
val = CachedArtifactDataset(file=val_path)

In [5]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

#autheticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient.from_config(
    credential=credential,
    path="config.json",
)

Found the config file in: config.json


In [6]:
import azure.ai.ml._artifacts._artifact_utilities as artifact_utils

data_asset_model = ml_client.data.get("artifactory_output", version="156")
artifact_utils.download_artifact_from_aml_uri(uri=data_asset_model.path, destination="./checkpoints_transformer/", datastore_operation=ml_client.datastores)
# paths_mask = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/notebooks/checkpoints_transformer/WindowLinearDetector_mask_639.28K_28-05-2024_07:50:53/epoch=0-step=15500.ckpt" # Mask CNN Trans
# mask_CNNTrans_detector = WindowTransformerDetector.load_from_checkpoint(paths_mask).cpu()

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy 'https://m3mlopssadev.blob.core.windows.net/azureml-blobstore-206414f2-5a5c-4209-8dbe-6d0e233cd920/artifactory/transformer/' './checkpoints_transformer/' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy 'https://m3mlopssadev.blob.core.windows.net/azureml-blobstore-206414f2-5a5c-4209-8dbe-6d0e233cd920/artifactory/transformer/' './checkpoints_transformer/' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.


'./checkpoints_transformer/'

In [5]:
paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/ablation_fpboost/SW_FCNTrans_fpbosst_0.ckpt" # SW FCN
adaFCNTrans_0 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/ablation_fpboost/SW_FCNTrans_fpboost_01.ckpt" # SW FCN
adaFCNTrans_0_1 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/ablation_fpboost/SW_FCNTrans_fpboost_02.ckpt" # SW FCN
adaFCNTrans_0_2 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/ablation_fpboost/SW_FCNTrans_fpboost_03.ckpt" # SW FCN
adaFCNTrans_0_3 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/ablation_fpboost/SW_FCNTrans_fpboost_04.ckpt" # SW FCN
adaFCNTrans_0_4 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

paths_SW = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/models/ablation_fpboost/SW_FCNTrans_fpboost_05.ckpt" # SW FCN
adaFCNTrans_0_5 = SlidingWindowTransformerDetector.load_from_checkpoint(paths_SW).cpu()

SW_detectors = [adaFCNTrans_0.eval(), adaFCNTrans_0_1.eval(), adaFCNTrans_0_2.eval(), adaFCNTrans_0_3.eval(), adaFCNTrans_0_4.eval(), adaFCNTrans_0_5.eval()]

/opt/miniconda/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:197: UserWarning: Attribute 'act_fct' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['act_fct'])`.
  rank_zero_warn(


In [6]:
def baseline_detector(input: torch.Tensor) -> int:   
    input.squeeze(0)
    prediction = 0

    center = int(input.shape[1]/2)
    # flag points with very high increment as artifact
    # Calculate increments by subtracting the tensor shifted by one from the original tensor
    increments = (input[0][1:] - input[0][:-1]).abs()
    mean_increment = torch.mean(increments)
    std_increment = torch.std(increments)

    if increments[center-1] > (mean_increment + 3*std_increment):
        prediction = 1
    
    return prediction

## Predictions on Validation set for threshold calculation with fbeta score

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error, confusion_matrix
import pandas as pd

all_predictions_valSet = pd.DataFrame(columns=['Detector_id', 'predictions'])
index = 0
gt = list()

for detector in SW_detectors:

    preds = list()

    for sample in val:
        example = sample["data"]
        window  = detector.window
        length  = len(example)

        # add artifact to data
        example_data = torch.tensor(example + sample["artifact"])

        # set detector to evaluation mode
        detector.eval()
        # make prediction and insert into prediction
        prediction = detector(example_data.unsqueeze(0))

        # update count
        preds = preds + [prediction.numpy()]

        if index == 0 :
            gt = gt + [sample["label"]]
    

    new_row = pd.DataFrame([{
        'Detector_id': index +1,
        'predictions': preds
    }])

    all_predictions_valSet = pd.concat([all_predictions_valSet,new_row])
    index = index +1
    print(len(all_predictions_valSet))

all_predictions_valSet = pd.concat([
    all_predictions_valSet,
    pd.DataFrame([{
        'Detector_id': 0,
        'predictions': gt
    }])
    ])


1
2


/opt/miniconda/lib/python3.10/site-packages/torch/nn/modules/conv.py:303: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:883.)
  return F.conv1d(input, weight, bias, self.stride,


3
4
5
6


In [10]:
from sklearn.metrics import fbeta_score

gt = all_predictions_valSet.iloc[-1]["predictions"]
metrics = pd.DataFrame()
for index, row in all_predictions_valSet.iterrows():

    max_fbeta = 0

    for threshold in np.linspace(0,  1,  100):
        predictions = np.where(np.array(row['predictions']) > threshold, 1, 0)

        fbeta = fbeta_score(gt, predictions, average='macro', beta=0.5)

        if (fbeta > max_fbeta):
            max_fbeta = fbeta
            best_threshold_fbeta = threshold

    predictions = np.where(np.array(row['predictions']) > best_threshold_fbeta, 1, 0)

    tn, fp, fn, tp = confusion_matrix(gt, predictions, labels=[0, 1]).ravel()

    metric = pd.DataFrame([{
        'index': index,
        'detector': f"detector{row['Detector_id']}",
        'threshold': best_threshold_fbeta,
        'fbeta_score': fbeta_score(gt, predictions, beta=0.5),
        'accuracy': accuracy_score(gt, predictions),
        'precision': precision_score(gt, predictions),
        'recall': recall_score(gt, predictions),
        'mse': mean_squared_error(gt, predictions), 
        'tn': tn,
        'fp': fp, 
        'fn': fn, 
        'tp': tp
    }])

    metrics = pd.concat([metrics, metric])


In [11]:
thresholds = list(metrics["threshold"])

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error, confusion_matrix
import pandas as pd

all_predictions = pd.DataFrame(columns=['Detector_id', 'predictions'])
index = 1
gt = list()
preds_baseline = list()

for sample in test:
    example = sample["data"]
    stride  = 64
    window  = test_width
    length  = len(example)

    example_data = torch.tensor(example + sample["artifact"])
    prediction_baseline = baseline_detector(example_data.unsqueeze(0))
    preds_baseline = preds_baseline + [prediction_baseline]

    gt = gt + [sample["label"]]

all_predictions = pd.concat([
    all_predictions,
    pd.DataFrame([{
        'Detector_id': index,
        'predictions': preds_baseline
    }])
    ], ignore_index=True)

index = 2

for detector in SW_detectors:

    preds = list()

    for sample in test:
        example = sample["data"]
        window  = detector.window
        length  = len(example)

        # add artifact to data
        example_data = torch.tensor(example + sample["artifact"])

        # set detector to evaluation mode
        detector.eval()
        # make prediction and insert into prediction
        prediction = detector(example_data.unsqueeze(0))

        # update count
        preds = preds + [prediction.numpy()]
    

    new_row = pd.DataFrame([{
        'Detector_id': index,
        'predictions': preds
    }])

    all_predictions = pd.concat([all_predictions,new_row], ignore_index=True)
    index = index +1
    print(len(all_predictions))


all_predictions = pd.concat([
    all_predictions,
    pd.DataFrame([{
        'Detector_id': 0,
        'predictions': gt
    }])
    ], ignore_index=True)



2
3


/opt/miniconda/lib/python3.10/site-packages/torch/nn/modules/conv.py:303: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:883.)
  return F.conv1d(input, weight, bias, self.stride,


4
5
6
7


In [13]:
thresholds

[0.4646464646464647,
 0.36363636363636365,
 0.25252525252525254,
 0.26262626262626265,
 0.48484848484848486,
 0.21212121212121213,
 0.0]

In [14]:
thresholds = [0] + thresholds

In [10]:
thresholds = [0, 0.464, 0.363, 0.252, 0.262, 0.484, 0.212, 0]

In [11]:
from sklearn.metrics import fbeta_score

gt = all_predictions.iloc[-1]["predictions"]
metrics = pd.DataFrame()
for index, row in all_predictions.iterrows():

    predictions = np.where(np.array(row['predictions']) > thresholds[index], 1, 0)

    tn, fp, fn, tp = confusion_matrix(gt, predictions, labels=[0, 1]).ravel()

    metric = pd.DataFrame([{
        'index': index,
        'detector': f"detector{row['Detector_id']}",
        'threshold': thresholds[index],
        'fbeta_score': fbeta_score(gt, predictions, beta=0.5),
        'accuracy': accuracy_score(gt, predictions),
        'precision': precision_score(gt, predictions),
        'recall': recall_score(gt, predictions),
        'mse': mean_squared_error(gt, predictions), 
        'tn': tn,
        'fp': fp, 
        'fn': fn, 
        'tp': tp
    }])

    metrics = pd.concat([metrics, metric])


In [12]:
metrics

,index,detector,threshold,fbeta_score,accuracy,precision,recall,mse,tn,fp,fn,tp
0,0,detector1,0.000,0.974131,0.966797,0.979675,0.952569,0.033203,1016,20,48,964
0,1,detector2,0.464,0.985772,0.976074,0.992835,0.958498,0.023926,1029,7,42,970
0,2,detector3,0.363,0.980431,0.977051,0.982983,0.970356,0.022949,1019,17,30,982
0,3,detector4,0.252,0.993512,0.984375,1.000000,0.968379,0.015625,1036,0,32,980
0,4,detector5,0.262,0.983157,0.974121,0.989785,0.957510,0.025879,1026,10,43,969
0,5,detector6,0.484,0.983157,0.974121,0.989785,0.957510,0.025879,1026,10,43,969
0,6,detector7,0.212,0.980509,0.978516,0.982072,0.974308,0.021484,1018,18,26,986
0,7,detector0,0.000,1.000000,1.000000,1.000000,1.000000,0.000000,1036,0,0,1012


In [18]:
metrics

,index,detector,threshold,fbeta_score,accuracy,precision,recall,mse,tn,fp,fn,tp
0,0,detector1,0.000000,0.974131,0.966797,0.979675,0.952569,0.033203,1016,20,48,964
0,1,detector2,0.464646,0.986145,0.975586,0.993840,0.956522,0.024414,1030,6,44,968
0,2,detector3,0.363636,0.983508,0.977539,0.987879,0.966403,0.022461,1024,12,34,978
0,3,detector4,0.252525,0.991837,0.980469,1.000000,0.960474,0.019531,1036,0,40,972
0,4,detector5,0.262626,0.983169,0.970215,0.992746,0.946640,0.029785,1029,7,54,958
0,5,detector6,0.484848,0.982724,0.973145,0.989765,0.955534,0.026855,1026,10,45,967
0,6,detector7,0.212121,0.986683,0.979492,0.991886,0.966403,0.020508,1028,8,34,978
0,7,detector0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1036,0,0,1012


In [31]:
import pandas as pd

test_width = 512
real_data_francois = pd.read_pickle('/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/real/francois_normalized_dataset.pickle') 

In [13]:
real_data_df = pd.read_pickle('/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/real/normalized_deviation_updated_TEST.pickle') 
ground_truth = pd.read_csv('/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/gt_changes_only_relabeled_200K.csv')

In [14]:
ground_truth.drop(columns=["Unnamed: 0"], inplace=True)

In [15]:
ground_truth[:493].sum()/len(ground_truth[:493])

position    136012.549696
gt               0.409736
dtype: float64

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error, confusion_matrix
import pandas as pd

all_predictions_real = pd.DataFrame(columns=['Detector_id', 'predictions'])
dist = test_width // 2

index = 1
gt = list()
preds_baseline = list()

for index, row in ground_truth[:494].iterrows():
    example_data = torch.tensor(real_data_df[0][int(row["position"]-dist) : int(row["position"]+dist)])
    
    prediction_baseline = baseline_detector(example_data.unsqueeze(0))
    preds_baseline = preds_baseline + [prediction_baseline]

    gt = gt + [row["gt"]]

all_predictions_real = pd.concat([
    all_predictions_real,
    pd.DataFrame([{
        'Detector_id': index,
        'predictions': preds_baseline
    }])
    ], ignore_index=True)

index = 2

for detector in SW_detectors:

    preds = list()

    for index, row in ground_truth[:494].iterrows():
        example_data = torch.tensor(real_data_df[0][int(row["position"]-dist) : int(row["position"]+dist)])
        # make prediction and insert into prediction
        prediction = detector(example_data.unsqueeze(0))

        # update count
        preds = preds + [prediction.numpy()] 

    new_row = pd.DataFrame([{
        'Detector_id': index ,
        'predictions': preds
    }])

    all_predictions_real = pd.concat([all_predictions_real,new_row], ignore_index=True)
    index = index +1
    print(len(all_predictions_real))


all_predictions_real = pd.concat([
    all_predictions_real,
    pd.DataFrame([{
        'Detector_id': 0,
        'predictions': gt
    }])
    ], ignore_index=True)

2
3
4
5
6
7


In [17]:
all_predictions_real.drop(all_predictions_real.tail(1).index,inplace=True)

In [18]:
all_predictions_real

,Detector_id,predictions
0,493,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,493,"[0.28180608, 0.10264383, 0.17861938, 0.2319259..."
2,493,"[0.4161924, 0.5848775, 0.28398928, 0.31664792,..."
3,493,"[0.0940011, 0.20374922, 0.15312523, 0.17135988..."
4,493,"[0.59705216, 0.34151748, 0.34851986, 0.4337693..."
5,493,"[0.51428175, 0.42112103, 0.3543118, 0.4821462,..."
6,493,"[0.30849463, 0.26142, 0.15335159, 0.21993889, ..."


In [19]:
from sklearn.metrics import fbeta_score

best_threshold_fbeta = 0.5

gt_real = ground_truth[:494]["gt"]
metrics_real = pd.DataFrame()

for index, row in all_predictions_real.iterrows():

    predictions = np.where(np.array(row['predictions']) > thresholds[index], 1, 0)
    indices_tp = [i for i, val in enumerate(predictions[gt_real==1] == 1) if val]
    indices_fn = [i for i, val in enumerate(predictions[gt_real==1] == 0) if val]

    tn, fp, fn, tp = confusion_matrix(gt_real, predictions, labels=[0, 1]).ravel()

    metric = pd.DataFrame([{
        'index': index,
        'detector': f"Detector{row['Detector_id']}",
        'threshold': thresholds[index],
        'fbeta_score': fbeta_score(gt_real, predictions, beta=0.5),
        'accuracy': accuracy_score(gt_real, predictions),
        'precision': precision_score(gt_real, predictions),
        'recall': recall_score(gt_real, predictions),
        'mse': mean_squared_error(gt_real, predictions), 
        'tn': tn,
        'fp': fp, 
        'fn': fn, 
        'tp': tp, 
        'indices_tp': indices_tp, 
        'indices_fn': indices_fn
    }])

    metrics_real = pd.concat([metrics_real, metric])


/opt/miniconda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
metrics_real

,index,detector,threshold,fbeta_score,accuracy,precision,recall,mse,tn,fp,fn,tp,indices_tp,indices_fn
0,0,Detector493,0.000,0.000000,0.591093,0.000000,0.000000,0.408907,292,0,202,0,[],"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
0,1,Detector493,0.464,0.679443,0.718623,0.838710,0.386139,0.281377,277,15,124,78,"[3, 6, 8, 9, 10, 12, 15, 16, 20, 21, 22, 23, 2...","[0, 1, 2, 4, 5, 7, 11, 13, 14, 17, 18, 19, 25,..."
0,2,Detector493,0.363,0.708419,0.759109,0.715026,0.683168,0.240891,237,55,64,138,"[0, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, ...","[1, 2, 5, 14, 26, 27, 31, 38, 39, 41, 48, 56, ..."
0,3,Detector493,0.252,0.464481,0.645749,0.829268,0.168317,0.354251,285,7,168,34,"[5, 15, 19, 21, 35, 42, 46, 52, 54, 58, 63, 69...","[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
0,4,Detector493,0.262,0.748441,0.789474,0.757895,0.712871,0.210526,246,46,58,144,"[0, 1, 3, 4, 6, 8, 9, 10, 11, 12, 14, 15, 17, ...","[2, 5, 7, 13, 16, 18, 27, 31, 39, 41, 45, 48, ..."
0,5,Detector493,0.484,0.720000,0.751012,0.788321,0.534653,0.248988,263,29,94,108,"[0, 3, 8, 9, 10, 12, 15, 16, 17, 19, 20, 21, 2...","[1, 2, 4, 5, 6, 7, 11, 13, 14, 18, 27, 31, 37,..."
0,6,Detector493,0.212,0.717017,0.771255,0.710900,0.742574,0.228745,231,61,52,150,"[0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[1, 5, 47, 48, 49, 57, 60, 61, 76, 81, 82, 83,..."
